# Solving combinatorial optimization problems.
When solving an instance of a combinatorial optimization problem, one is generally interested in an object from a finite or, possibly, a countable infinite set that satisfies certain conditions or constraints. Such an object can be an integer, a set, a permutation, or a graph.
This notebook shows how to define and solve an instance of a travelling salesman problem (TSP).

## TSP.
The TSP is a popular NP-hard combinatorial problem, of high importance in theoretical and applied computer science and operations research. It inspired several other important problems, such as vehicle routing and travelling purchaser. In simple terms, the travelling salesman must visit every city in a given region exactly once and then return to the starting point. The problem proposes the following question: given the cost of travel between all the cities, how should the itinerary be planned to minimize the total cost of the entire tour.

<img src="https://i.pinimg.com/originals/b0/95/4c/b0954c93b3c37876e9b54736453a9d5c.jpg" alt="Drawing" style="width: 400px;"/>

# 1. Create problems' instances.

Loads the necessary classes and functions.

In [16]:
# Imports PyTorch
import torch
# Imports problems
from gpol.problems.tsp import TSP
from gpol.utils.utils import travel_distance
# Imports metaheuristics 
from gpol.algorithms.random_search import RandomSearch
from gpol.algorithms.local_search import HillClimbing
from gpol.algorithms.local_search import SimulatedAnnealing
from gpol.algorithms.genetic_algorithm import GeneticAlgorithm
# Imports operators
from gpol.operators.initializers import rnd_vshuffle, rnd_mshuffle
from gpol.operators.selectors import prm_tournament
from gpol.operators.variators import partially_mapped_xo, prm_iswap_mnt

Creates an instance of ``TSP``. The search space (*S*) of an instance of ``TSP`` consists of the following key-value pairs:
- ``"distances"`` is a $n$x$n$ tensor of type ``torch.float`` which represents the distances' matrix for the underlying problem. The matrix can be either symmetric or asymmetric. A given row $i$ in the matrix represents the set of distances between $i$, as being the origin, and the $n$ possible destinations (including $i$ itself);
- ``"origin"``: an integer value representing the origin, i.e., the point from where the *travelling salesman* departs.

In [17]:
# Defines the processing device and random state 's seed
device, seed = "cpu", 0  # 'cuda' if torch.cuda.is_available() else 'cpu', 0
# Characterizes the problem: distance matrix (from https://developers.google.com/optimization/routing/tsp)
dist_mtrx = torch.tensor([[0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
                [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
                [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
                [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
                [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
                [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
                [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
                [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
                [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
                [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
                [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
                [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
                [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0]], dtype=torch.float, device=device)

# Creates the search space
sspace_tsp = {"distances": dist_mtrx, "origin": 3}
# Creates an instance of TSP
pi_tsp = TSP(sspace=sspace, ffunction=travel_distance, min_=True)

# 2. Choose and parametrize the algorithms.

## 2.1. Random search (RS).
The random search (RS) can be seen as thethe first rudimentary stochastic metaheuristic for problem-solving. Its strategy, far away from being *intelligent*, consists of randomly sampling $S$ for a given number of iterations. As such, the only search-parameter of an instance of ``RandomSearch`` is the initialization function (the ``initializer``). The function ``rnd_vshuffle`` is one of the numerous initializers for the single-point (SP) algorithms and it randomly shuffles *cities*' indexes (which are defined in the $S$ of the problem's instance).

The cell in below creates two dictionaries, called ``pars`` which stores algorithms' parameters for each problem type. Each key-value pair stores the algorithm's type and a dictionary of respective search-parameters for a given problem. 

In [18]:
# Defines single-point (SP) initializers for each problem
sp_init = rnd_vshuffle
# Defines RS's parameters
pars = {RandomSearch: {"initializer": sp_init}}

## 2.2. Hill climbing (HC).
The local search (LS) algorithms can be seen among the first intelligent search strategies that improve the functioning of the RS. They rely upon the concept of neighborhood which is explored at each iteration by sampling from $S$ a limited number of neighbors of the best-so-far solution. Usually, the LS algorithms are divided in two branches. In the first branch, called hill climbing (HC), or hill descent for the minimization problems, the best-so-far solution is replaced by its neighbor when the latter is at least as good as the former.

The cell in below adds ``HillClimbing`` to ``pars``. Note that, unlike it was for ``RandomSearch``, an instance of ``HillClimbing`` also requires the specification of a neighbor-generation function (``"nh_function"``) and the neighborhood's size (``"nh_size"``). In this example, the so-called *swap-mutation* is used for the TSP, with a probability of swapping a random indexes' pair equal to $0.3$\%. Note that the very same initialization function is used for both ``RandomSearch`` and ``HillClimbing``. 

In [19]:
# Defines the size of the neighborhood 
nh_size = 100
# Defines neighbor-generation function with the respective parameters
nh_function = prm_iswap_mnt(prob=0.3)
# Defines HC's parameters
pars[HillClimbing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size}

## 2.3. Simulated annealing (SA).
The second branch, called simulated annealing (SA), extends HC by formulating a non-negative probability of replacing the best-so-far solution by its neighbor when the latter is worse. Traditionally, such a probability is small and decreases as the search advances. The strategy adopted by SA is especially useful when the search is prematurely tagnated at a locally sub-optimal point in $S$.

The cell in below adds ``SimulatedAnnealing`` to ``pars``. 

In [22]:
# Defines SA's parameters
pars[SimulatedAnnealing] = {"initializer": sp_init, "nh_function": nh_function, "nh_size": nh_size, "control": 1.0, "update_rate": 0.9}

## 2.4. Genetic algorithm (GA).
Based on the number of candidate solutions they handle at each step, the metaheuristics can be categorized into single-point (SP) and population-based (PB) approaches. 
The search procedure in the SP metaheuristics is generally guided by the information provided by a single candidate solution from $S$, usually the best-so-far solution, that is gradually evolved in a well-defined manner in hope to find the global optimum. The abovementioned HC and SA are examples of SP metaheuristics as the search is performed by exploring the neighborhood $N(i)$, where $i$ is the current best solution. Contrarily, the search procedure in PB metaheuristics is generally guided by the information shared by a set of candidate solutions and the exploitation of its collective behavior in different ways. In abstract terms, one can say that every PB metaheuristics shares, at least, the following two features: an object representing the set of simultaneously exploited candidate solutions (i.e., the population), and a procedure to *move* them across $S$.

Genetic Algorithm (GAs) is a meta-heuristic introduced by J. Holland which was strongly inspired by Darwin's theory of evolution by means of natural selection. Conceptually, the algorithm starts with a random-like population of candidate solutions (called *chromosomes*). Then, by mimicking the natural selection and genetically inspired variation operators, such as the crossover and the mutation, the algorithm breeds a population of the next-generation candidate solutions (called the *offspring population*), that replaces the previous population (a.k.a. the *parent population*). This procedure is iterated until reaching some stopping criteria, like a maximum
number of iterations (also called *generations*).

The cell in below adds ``GeneticAlgorithm`` to ``pars``. It uses a slightly different initializer, specially designed to efficiently initialize PB metaheuristics, called ``rnd_mshuffle``, that shuffles a collection of cities' vectors organized in a matrix. Note that the mutation function is the neighbor-generation function, that was used for the aforementioned LS algorithms, and the population's size is equivalent to the neighborhood's size; this is done to foster the equivalency between LS and PB metaheuristics. Finally, the crossover is partially mapped, the elite is always a member of the population and parents' reproduction is enabled (``elitism=True`` and ``reproduction=True``).

In [23]:
# Defines a population-based (PB) initializer
pb_init = rnd_mshuffle
# Defines GA's parameters
pars[GeneticAlgorithm] = {"pop_size": nh_size, "initializer": pb_init, "selector": prm_tournament(pressure=0.05), "mutator": nh_function, 
                          "crossover": partially_mapped_xo, "p_m": 0.3, "p_c": 0.7, "elitism": True, "reproduction": True}

# 3. Executes the experiment.

Note that *many* parameters and functions are shared across different algorithms in the experiment. This allows to increase the control and comparability between different algorithmic approaches when solving a given problem's instance.

In [7]:
for isa_type, isa_pars in pars.items():
    print(isa_type)
    for p_name, p_val in isa_pars.items():
        print("\t", p_name, p_val)

<class 'gpol.algorithms.random_search.RandomSearch'>
	 initializer <function rnd_vshuffle at 0x00000292A35428B0>
<class 'gpol.algorithms.local_search.HillClimbing'>
	 initializer <function rnd_vshuffle at 0x00000292A35428B0>
	 nh_function <function prm_iswap_mnt.<locals>.iswap_mnt at 0x00000292A3542F70>
	 nh_size 100
<class 'gpol.algorithms.local_search.SimulatedAnnealing'>
	 initializer <function rnd_vshuffle at 0x00000292A35428B0>
	 nh_function <function prm_iswap_mnt.<locals>.iswap_mnt at 0x00000292A3542F70>
	 nh_size 100
	 control 1
	 update_rate 0.95
<class 'gpol.algorithms.genetic_algorithm.GeneticAlgorithm'>
	 pop_size 100
	 initializer <function rnd_mshuffle at 0x00000292A3542940>
	 selector <function prm_tournament.<locals>.tournament at 0x00000292A3568E50>
	 mutator <function prm_iswap_mnt.<locals>.iswap_mnt at 0x00000292A3542F70>
	 crossover <function partially_mapped_xo at 0x00000292A3538B80>
	 p_m 0.5
	 p_c 0.5
	 elitism True
	 reproduction False


Defines the computational resources for the experiment: the number of iterations.

In [24]:
n_iter = 30

Loops the afore-defined ``pars`` dictionary containing algorithms' and the underlying parameters. Note that besides algorithm-specific parameters, the constructor of an instance of a search algorithm also receives the random state to initialize a pseudorandom number generator (called ``seed``), and the specification of the processing ``device`` (either CPU or GPU).

The ``solve`` method has the same signature for all the search algorithms and, in this example, includes the following parameters: 
-  ``n_iter``: number of iterations to conduct the search;
-  ``tol``: minimum required fitness improvement for ``n_iter_tol`` consecutive iterations to continue the search. When the fitness is not improving by at least ``tol`` for ``n_iter_tol`` consecutive iterations, the search will be automatically interrupted;
-  ``n_iter_tol``: maximum number of iterations to not meet ``tol`` improvement;
-  ``verbose``: verbosity's detail-level;
-  ``log``: log-files' detail-level (if exists).

In [25]:
for isa_type, isa_pars in pars.items():
    isa = isa_type(pi=pi, **isa_pars, seed=seed, device=device)
    # n_iter*pop_size if isinstance(isa, RandomSearch) else n_iter  # equivalency for the RS
    isa.solve(n_iter=n_iter, tol=20, n_iter_tol=5, verbose=2, log=0)
    print("Algorithm: {}".format(isa_type.__name__))
    print("Best solution's fitness: {:.3f}".format(isa.best_sol.fit))
    print("Best solution:", isa.best_sol.repr_, end="\n\n")

-------------------------------------------------
           |           Best solution            |
-------------------------------------------------
Generation   Length   Fitness              Timing
0            12       16524                 0.002
1            12       14459                 0.000
2            12       14459                 0.000
3            12       14459                 0.000
4            12       14459                 0.000
5            12       14459                 0.000
6            12       14459                 0.000
Algorithm: RandomSearch
Best solution's fitness: 14459.000
Best solution: tensor([ 2,  4, 12,  5,  1, 10,  7,  0,  6,  8,  9, 11])

--------------------------------------------------------------------------------------
           |           Best solution              |           Neighborhood           |
--------------------------------------------------------------------------------------
Generation | Length   Fitness              Timing | AVG F

C:\Miniconda\envs\gpol_test\lib\site-packages\gpol\operators\variators.py:459: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  o1[idxs_out_[out]] = o2[idxs_in_][(o1[idxs_out_[out]] == o1[idxs_in_]).nonzero()[0]]


2          | 12       10735                 0.067 | 13817.3                   1985.15
3          | 12       10206                 0.050 | 13059.9                   1997.49
4          | 12       9227                  0.049 | 12195                     1832.25
5          | 12       9227                  0.048 | 12315.5                   2283.86
6          | 12       9148                  0.041 | 11524.4                   2226.34
7          | 12       9148                  0.038 | 11252.2                   2483.42
8          | 12       8785                  0.042 | 11058.9                   2378.56
9          | 12       8455                  0.039 | 10690.3                   2592.47
10         | 12       8455                  0.035 | 10737.7                   2904.01
11         | 12       8455                  0.038 | 10860.1                   3118.62
12         | 12       8455                  0.038 | 9997.22                   2348.86
13         | 12       8455                  0.032 | 10